In [ ]:
%run "NN101_helpers.ipynb"

# Olivetti Faces Dataset

In [ ]:
# list out dataset
fig = plt.figure()
gs = gridspec.GridSpec(5, 8, wspace=0.0)
ax = [plt.subplot(gs[i]) for i in range(40)]
gs.update(hspace=0)
for i in range(0,399,10):
    if has_glasses[i] >= 0:
        ax[i/10].imshow(fd.images[i], cmap='gist_gray')
    ax[i/10].axis('off')


In [ ]:
show_face_row(16)
show_face_row(18)

In [ ]:
imshow(fd.images[1*10+2], cmap="gist_gray")

In [ ]:
print "Matrix dimensions:"
print fd.images[16*10].shape
for i in range(0, fd.images[16*10].shape[0], 3):
    for j in range(0, fd.images[16*10].shape[1], 3):
        print np.round(fd.images[16*10][i,j],2),
    print ""

# Glasses or Not?
#### How to train a Neural Net to recognize glasses

In [ ]:
# Setup targets
print flattened_data.shape
print target_glasses

# Setup up Neural Network for glasses/not

In [ ]:
# are 5 iterations and 1 hidden neuron going to be enough?
nnmlp = MLPClassifier(
    activation='tanh',
    hidden_layer_sizes=(1),
    solver='sgd',
    verbose=False, 
    tol=1e-5, 
    max_iter=5)

In [ ]:
nncls = nnmlp.fit(flattened_data[gtraining], target_glasses[gtraining])

In [ ]:
nncls.score(flattened_data[gtraining], target_glasses[gtraining])

In [ ]:
print "Number of layers: %s\nOutput activations: %s" % (nncls.n_layers_, nncls.n_outputs_)

# A look inside the Neural Network

### the raw data

In [ ]:
for i, layer in enumerate(nncls.coefs_):
    print "Layer:", i
    print "  intercept:", nncls.intercepts_[i]
    print "  Coefs:", layer.shape 
    print layer

### a visual display of the neurons

In [ ]:
print_neurons(nncls, layer = 0)
print_neurons(nncls, layer = 1)
#print_neurons(nncls, layer = 2)

In [ ]:
nncls.predict_proba(flattened_data[~gtraining])

# Compare logistic regression on glasses/not

In [ ]:
logit = LogisticRegression(tol=1e-5, max_iter=5)

In [ ]:
print logit.fit(flattened_data[gtraining], target_glasses[gtraining])


In [ ]:
print "Layer:", 0
print "  intercept:", logit.intercept_
print "  Coefs:", logit.coef_.shape
print "  ", logit.coef_.transpose()

In [ ]:
logistic_activations = imshow(
    logit.coef_.reshape((64,64)), cmap='gist_gray', interpolation="nearest")

# ROC Curves on glasses/not

In [ ]:
tpr_l, fpr_l, thh_l = return_roc_curve(flattened_data, target_glasses, logit, ~gtraining)
tpr_n, fpr_n, thh_n = return_roc_curve(flattened_data, target_glasses, nncls, ~gtraining)
plot_roc(flattened_data, target_glasses, logit, nncls, gtraining)

# Learning Curves and Benchmarking on glasses/not

In [ ]:
nnmlp = MLPClassifier(
    activation='tanh',
    hidden_layer_sizes=(36,4),
    solver='sgd',
    verbose=False, 
    tol=1e-4, 
    max_iter=200)
logit = LogisticRegression(tol=1e-4, max_iter=200)
nn_curve = np.array(learning_curve(
    model=nnmlp, data=flattened_data, target=has_glasses, test_set=~gtraining, rs=1000))
lg_curve = np.array(learning_curve(
    model=logit, data=flattened_data, target=has_glasses, test_set=~gtraining, rs=1000))
#for i in [1,2]:
#    nn_curve = nn_curve + np.array(learning_curve(
#        model=nnmlp, data=flattened_data, target=has_glasses, test_set=~gtraining, rs=i))
#    lg_curve = lg_curve + np.array(learning_curve(
#        model=logit, data=flattened_data, target=has_glasses, test_set=~gtraining, rs=i))


In [ ]:
np.array(nn_curve) + np.array(nn_curve)

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 2)
ax = [plt.subplot(gs[0]), plt.subplot(gs[1])]
x = ax[0].plot(nn_curve[0], nn_curve[1], "blue", nn_curve[0], nn_curve[2], "b--", zorder=10)
y = ax[1].plot(lg_curve[0], lg_curve[1], "green", lg_curve[0], lg_curve[2], "g--", zorder=10)
plt.show()

plt.plot(nn_curve[0], nn_curve[3], "blue", lg_curve[0], lg_curve[3], "green")
plt.show()

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(1, 2)
ax = [plt.subplot(gs[0]), plt.subplot(gs[1])]
x = ax[0].plot(nn_curve[0], nn_curve[1], "blue", nn_curve[0], nn_curve[2], "b--", zorder=10)
y = ax[1].plot(lg_curve[0], lg_curve[1], "green", lg_curve[0], lg_curve[2], "g--", zorder=10)
plt.show()

plt.plot(nn_curve[0], nn_curve[3], "blue", lg_curve[0], lg_curve[3], "green")
plt.show()

# Beards and Glasses
#### Lets complicate things a little

In [ ]:
glasses_and_beard = has_beard * has_glasses

In [ ]:
glasses_and_beard[gtraining]

In [ ]:
nnmlp = MLPClassifier(
    activation='tanh', 
    hidden_layer_sizes=(16,8), 
    solver='sgd', verbose=True, 
    tol=1e-5, random_state=2000, max_iter=100, learning_rate_init=1e-4)

In [ ]:
nncls = nnmlp.fit(flattened_data[gtraining], glasses_and_beard[gtraining])

In [ ]:
print "Number of layers: %s\nOutput activations: %s" % (nncls.n_layers_, nncls.n_outputs_)

In [ ]:
print_neurons(nncls, layer = 0)
print_neurons(nncls, layer = 1)
print_neurons(nncls, layer = 2)

# Compare logistic regression for glasses and beards

In [ ]:
logit = LogisticRegression(tol=1e-8, max_iter=100)

In [ ]:
print logit.fit(flattened_data[gtraining], glasses_and_beard[gtraining])


In [ ]:
print "Layer:", 0
print "  intercept:", logit.intercept_
print "  Coefs:", logit.coef_.shape
print "  ", logit.coef_.transpose()

In [ ]:
logistic_activations = imshow(logit.coef_.reshape((64,64)), cmap='gist_gray', interpolation="nearest")

# ROC Curve for glasses and beards

In [ ]:
tpr_l, fpr_l, thh_l = return_roc_curve(flattened_data, glasses_and_beard, logit, ~gtraining)
tpr_n, fpr_n, thh_n = return_roc_curve(flattened_data, glasses_and_beard, nncls, ~gtraining)
plot_roc(flattened_data, target_glasses, logit, nncls, gtraining)

In [ ]:
for i, layer in enumerate(nncls.coefs_):
    print "Layer:", i
    print "  intercept:", nncls.intercepts_[i]
    print "  Coefs:", layer.shape 
    print layer

# Facial Recognition
#### Lets complicate a bit further

In [ ]:
individuals[itraining]

### Specification of hidden layers can have a big effect on a neural nets ability to converge well 

In [ ]:
nnmlp = MLPClassifier(
    activation='tanh', 
    hidden_layer_sizes=(8,4), 
    solver='lbfgs', verbose=True, 
    tol=1e-6, 
    random_state=2001, 
    max_iter=15000, 
    learning_rate_init=1e-3)

In [ ]:
nncls = nnmlp.fit(flattened_data[itraining], individuals[itraining])

In [ ]:
print "Number of layers: %s\nOutput activations: %s" % (nncls.n_layers_, nncls.n_outputs_)

In [ ]:
for i, layer in enumerate(nncls.coefs_):
    print "Layer:", i
    print "  intercept:", nncls.intercepts_[i]
    print "  Coefs:", layer.shape 
    print layer

In [ ]:
print_neurons(nncls, layer = 0)
print_neurons(nncls, layer = 1)

In [ ]:
list(np.round(nncls.predict_proba(flattened_data[~individuals]),2))

# Compare logistic regression for glasses and beards

In [ ]:
logit = LogisticRegression(max_iter=100, multi_class='multinomial', solver='lbfgs')

In [ ]:
print logit.fit(flattened_data[itraining], individuals[itraining])


In [ ]:
print "Layer:", 0
print "  intercept:", logit.intercept_
print "  Coefs:", logit.coef_.shape
print "  ", logit.coef_.transpose()

In [ ]:
logistic_activations = imshow(
    logit.coef_[2].reshape((64,64)), cmap='gist_gray', interpolation="nearest")
plt.show()
logistic_activations = imshow(
    logit.coef_[5].reshape((64,64)), cmap='gist_gray', interpolation="nearest")


# Accuracy comparison for individual facial recognition

In [ ]:
print logit.score(flattened_data, individuals)
print nncls.score(flattened_data, individuals)

# Neural Net Example with Continuous Target

In [ ]:
from sklearn.datasets import load_boston
housing = load_boston()
#help(housing)
print housing.target
print housing.data

In [ ]:
nnmlp = MLPRegressor(
    activation='identity', 
    hidden_layer_sizes=(4,2), 
    solver='lbfgs', verbose=True, 
    tol=1e-5, random_state=2000, max_iter=10000, learning_rate_init=1e-4)

In [ ]:
nncls = nnmlp.fit(housing.data, housing.target)

In [ ]:
np.mean((nncls.predict(housing.data)-housing.target)**2)

In [ ]:
linear = LinearRegression()
linear.fit(housing.data, housing.target)
np.mean((linear.predict(housing.data)-housing.target)**2)